# Day 7: 회귀분석 (1) - 모델 구축

**날짜**: 2025-07-10

**목표**:
- 다중 선형 회귀 모델 구축
- VIF (다중공선성) 검사
- 모델 평가 (R², Adjusted R², p-value)

In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

from utils import *

set_korean_font()

df = pd.read_csv(os.path.join(DATA_PATHS['processed'], 'integrated_data.csv'), encoding='utf-8-sig')
print(f"✅ Day 7: 회귀분석 - 데이터 로드 {df.shape}")

## 1. 회귀분석 설계

**종속변수**: 인구당_CCTV효과범죄율

**독립변수**:
- 인구당_방범용 (주 관심 변수)
- 인구밀도 (통제 변수)

In [ ]:
# 변수 설정
X_cols = ['인구당_방범용', '인구밀도']
y_col = '인구당_CCTV효과범죄율'

X = df[X_cols]
y = df[y_col]

print(f"독립변수: {X_cols}")
print(f"종속변수: {y_col}")
print(f"\n샘플 수: {len(df)}")

## 2. VIF (다중공선성 검사)

In [ ]:
# VIF 계산
vif_data = pd.DataFrame()
vif_data['변수'] = X_cols
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X_cols))]

print("VIF (Variance Inflation Factor):")
print(vif_data)
print("\n해석: VIF < 10 → 다중공선성 문제 없음")

## 3. 다중 선형 회귀 모델

In [ ]:
# statsmodels로 회귀분석 (p-value 확인용)
X_with_const = sm.add_constant(X)
model = sm.OLS(y, X_with_const).fit()

print("="*70)
print("다중 선형 회귀 결과")
print("="*70)
print(model.summary())

print(f"\n주요 결과:")
print(f"  R²: {model.rsquared:.4f}")
print(f"  Adjusted R²: {model.rsquared_adj:.4f}")
print(f"\n회귀계수:")
for var, coef, pval in zip(['절편'] + X_cols, model.params, model.pvalues):
    sig = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
    print(f"  {var}: {coef:.4f} (p={pval:.4f}) {sig}")

## 4. 결과 해석

In [ ]:
interpretation = f"""
📊 회귀분석 결과 해석:

1. 모델 적합도
   - R² = {model.rsquared:.4f}: 독립변수들이 종속변수 변동의 {model.rsquared*100:.1f}% 설명
   - Adjusted R² = {model.rsquared_adj:.4f}

2. 방범용 CCTV 효과
   - 계수: {model.params['인구당_방범용']:.4f}
   - p-value: {model.pvalues['인구당_방범용']:.4f}
   - 해석: {'통계적으로 유의미' if model.pvalues['인구당_방범용'] < 0.05 else '통계적으로 유의미하지 않음'}

3. 한계점
   - 역인과성: 범죄가 많은 곳에 CCTV를 더 설치했을 가능성
   - 단면 데이터: 시간적 인과관계 불명확
   - 누락 변수: 조명, 경찰 순찰 등 미포함
"""

print(interpretation)

# 파일 저장
with open(os.path.join(DATA_PATHS['reports'], 'day7_regression_result.txt'), 'w', encoding='utf-8') as f:
    f.write(interpretation)

In [ ]:
print("✅ Day 7 완료")